In [35]:
import numpy as np

# Set a random seed for reproducibility (optional)
np.random.seed(42)

image = np.random.randint(0, 255, size = (3,3,5))

image

array([[[102, 179,  92,  14, 106],
        [ 71, 188,  20, 102, 121],
        [210, 214,  74, 202,  87]],

       [[116,  99, 103, 151, 130],
        [149,  52,   1,  87, 235],
        [157,  37, 129, 191, 187]],

       [[ 20, 160, 203,  57,  21],
        [252, 235,  88,  48, 218],
        [ 58, 254, 169, 219, 187]]])

In [36]:
landtype = ['bare', 'water', 'crops', 'forest']

land_use = np.random.choice(landtype, size = (1, 3, 5))

land_use

array([[['forest', 'crops', 'water', 'water', 'crops'],
        ['water', 'crops', 'forest', 'crops', 'forest'],
        ['forest', 'bare', 'crops', 'bare', 'crops']]], dtype='<U6')

In [37]:
# vstack puts the land_use image at the bottom
# in rioxarray this is accomplished using xr.concat()
full_array = np.vstack([image, land_use])

full_array

array([[['102', '179', '92', '14', '106'],
        ['71', '188', '20', '102', '121'],
        ['210', '214', '74', '202', '87']],

       [['116', '99', '103', '151', '130'],
        ['149', '52', '1', '87', '235'],
        ['157', '37', '129', '191', '187']],

       [['20', '160', '203', '57', '21'],
        ['252', '235', '88', '48', '218'],
        ['58', '254', '169', '219', '187']],

       [['forest', 'crops', 'water', 'water', 'crops'],
        ['water', 'crops', 'forest', 'crops', 'forest'],
        ['forest', 'bare', 'crops', 'bare', 'crops']]], dtype='<U11')

In [50]:
from rasterio.plot import reshape_as_image, reshape_as_raster

# change from band, x, y to x, y, band
# then reshape into z columns, where z is the number of bands
image_array = reshape_as_image(full_array)

# convert all pixels to rows
ml_prep_array = image_array.reshape(-1, full_array.shape[0])

ml_prep_array

array([['102', '116', '20', 'forest'],
       ['179', '99', '160', 'crops'],
       ['92', '103', '203', 'water'],
       ['14', '151', '57', 'water'],
       ['106', '130', '21', 'crops'],
       ['71', '149', '252', 'water'],
       ['188', '52', '235', 'crops'],
       ['20', '1', '88', 'forest'],
       ['102', '87', '48', 'crops'],
       ['121', '235', '218', 'forest'],
       ['210', '157', '58', 'forest'],
       ['214', '37', '254', 'bare'],
       ['74', '129', '169', 'crops'],
       ['202', '191', '219', 'bare'],
       ['87', '187', '187', 'crops']], dtype='<U11')

In [41]:
# convert to X_train/y_train
X_train = ml_prep_array[:, :-1]
y_train = ml_prep_array[:, -1]

X_train, y_train

(array([['102', '116', '20'],
        ['179', '99', '160'],
        ['92', '103', '203'],
        ['14', '151', '57'],
        ['106', '130', '21'],
        ['71', '149', '252'],
        ['188', '52', '235'],
        ['20', '1', '88'],
        ['102', '87', '48'],
        ['121', '235', '218'],
        ['210', '157', '58'],
        ['214', '37', '254'],
        ['74', '129', '169'],
        ['202', '191', '219'],
        ['87', '187', '187']], dtype='<U11'),
 array(['forest', 'crops', 'water', 'water', 'crops', 'water', 'crops',
        'forest', 'crops', 'forest', 'forest', 'bare', 'crops', 'bare',
        'crops'], dtype='<U11'))

In [42]:
y_pred = np.random.choice(landtype, size = X_train.shape[0])

y_pred

array(['crops', 'bare', 'bare', 'crops', 'water', 'forest', 'bare',
       'forest', 'water', 'water', 'water', 'bare', 'water', 'bare',
       'water'], dtype='<U6')

In [43]:
# works, quick way of combining arrays length-wise
classified_image = np.c_[X_train, y_pred]

classified_image

array([['102', '116', '20', 'crops'],
       ['179', '99', '160', 'bare'],
       ['92', '103', '203', 'bare'],
       ['14', '151', '57', 'crops'],
       ['106', '130', '21', 'water'],
       ['71', '149', '252', 'forest'],
       ['188', '52', '235', 'bare'],
       ['20', '1', '88', 'forest'],
       ['102', '87', '48', 'water'],
       ['121', '235', '218', 'water'],
       ['210', '157', '58', 'water'],
       ['214', '37', '254', 'bare'],
       ['74', '129', '169', 'water'],
       ['202', '191', '219', 'bare'],
       ['87', '187', '187', 'water']], dtype='<U11')

In [53]:
# need to remember shape of initial image if you want to get it back

pred_image = classified_image.reshape(image.shape[1], image.shape[2], -1)

pred_image

array([[['102', '116', '20', 'crops'],
        ['179', '99', '160', 'bare'],
        ['92', '103', '203', 'bare'],
        ['14', '151', '57', 'crops'],
        ['106', '130', '21', 'water']],

       [['71', '149', '252', 'forest'],
        ['188', '52', '235', 'bare'],
        ['20', '1', '88', 'forest'],
        ['102', '87', '48', 'water'],
        ['121', '235', '218', 'water']],

       [['210', '157', '58', 'water'],
        ['214', '37', '254', 'bare'],
        ['74', '129', '169', 'water'],
        ['202', '191', '219', 'bare'],
        ['87', '187', '187', 'water']]], dtype='<U11')

In [54]:
# we got back to original image with new predicted values
reshape_as_raster(pred_image)

array([[['102', '179', '92', '14', '106'],
        ['71', '188', '20', '102', '121'],
        ['210', '214', '74', '202', '87']],

       [['116', '99', '103', '151', '130'],
        ['149', '52', '1', '87', '235'],
        ['157', '37', '129', '191', '187']],

       [['20', '160', '203', '57', '21'],
        ['252', '235', '88', '48', '218'],
        ['58', '254', '169', '219', '187']],

       [['crops', 'bare', 'bare', 'crops', 'water'],
        ['forest', 'bare', 'forest', 'water', 'water'],
        ['water', 'bare', 'water', 'bare', 'water']]], dtype='<U11')